<a href="https://colab.research.google.com/github/michaelmoschitto/CoverLetterGPT/blob/main/Promptclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=c5be5f2b4331a760b087c552336e3a1bf10e40b0f3413b88d4057f16688c4d68
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-06-01 00:41:17--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  21.4MB/s    in 5.6s    

2023-06-01 00:41:23 (14.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
NUM_TO_CAT = {0 : "Complete a sentence", 1 : "Read cover letter and answer questions", 2 : "Generate a cover letter with constraints"}
CAT_TO_NUM = {"Complete a sentence" : 0, "Read cover letter and answer questions" : 1, "Generate a cover letter with constraints" : 2}

In [14]:
import pandas as pd
data = {
    'prompt': [
        "Complete the sentence: I love to...",
        "Complete this statement: The best thing about summer is...",
        "Finish the following sentence: Life is like a...",
        "Complete this sentence",
        "Write the end of this sentence please: My name is ",
        "Write the end of this sentence please: I believe I am a great fit for the",
        "Write the end of this sentence please: Beyond my technical skills, my strong communication,",
        "Write the end of this sentence please: My technical experience, combined with a growth mindset and passion...",
        "Finish the following sentence: My technical experience, combined with a growth mindset and passion...",
        "Complete this phrase: My technical experience, combined with a growth mindset and passion...",
        "Complete this sentence: My technical experience, combined with a growth mindset and passion...",
        
        "Read the cover letter below and answer the questions.",
        "Please review the attached cover letter and answer the following questions.",
        "Answer the questions based on the cover letter provided.",
        "Answer these questions after reading this cover letter",
        "Review the cover letter below and answer the questions: ",
        "Please answer the following questions after review my cover letter",
        "Please answer the following questions after review my cover letter",
        "Please read and answer these questions based on my cover letter",
        "Read my cover letter. Next, answer these questions",
        "Review this cover letter. After, answer these questions:",
        "Answer the following question based on my cover letter",

        "Generate a cover letter expressing your interest in the position.",
        "Compose a cover letter for the job application with the given specifications.",
        "Write a cover letter demonstrating your skills and qualifications for the position.",
        "Write me a cover letter demonstrating my skills. these include: ",
        "Generate a cover letter and include these skills: ",
        "Write me a cover letter demonstrating the following skills for this job: ",
        "Compose me a cover letter that answers this call for employment: ",
        "Generate a cover letter responding to this job application: ",
        "Write a cover letter for the job posting and these qualifications: ",
        "Generate a cover letter responding to the following job application. Be sure to include these skills. ",
        "Write a cover letter for me. Include the following skills. Respond to the follow job posting: ",
    ],
    'target': [
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",

        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",


        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",

    ]
}

train_data = pd.DataFrame(data)
train_data["target"] = train_data["target"].apply(lambda x: CAT_TO_NUM[x])
train_data.head(5)

,prompt,target
0,Complete the sentence: I love to...,0
1,Complete this statement: The best thing about ...,0
2,Finish the following sentence: Life is like a...,0
3,Complete this sentence,0
4,Write the end of this sentence please: My name...,0


In [13]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [19]:
from sklearn.model_selection import train_test_split
train_prompts, val_prompts, train_labels, val_labels = train_test_split(list(train_data["prompt"].values), list(train_data["target"].values), test_size=.2, shuffle=True) 
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)
print(train_labels, train_prompts)

[1, 0, 0, 2, 1, 2, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 2, 1] ['Please review the attached cover letter and answer the following questions.', 'Complete this statement: The best thing about summer is...', 'Complete this sentence: My technical experience, combined with a growth mindset and passion...', 'Generate a cover letter and include these skills: ', 'Review the cover letter below and answer the questions: ', 'Write a cover letter for me. Include the following skills. Respond to the follow job posting: ', 'Finish the following sentence: Life is like a...', 'Generate a cover letter responding to the following job application. Be sure to include these skills. ', 'Write me a cover letter demonstrating my skills. these include: ', 'Complete the sentence: I love to...', 'Write the end of this sentence please: Beyond my technical skills, my strong communication,', 'Finish the following sentence: My technical experience, combined with a growth mindset and passion...', 'Rev

In [ ]:
# train_texts = train_texts[:50]
# val_texts = val_texts[:50]
# test_texts = test_texts[:50]

# train_labels = train_labels[:50]
# val_labels = val_labels[:50]
# test_labels = test_labels[:50]

In [16]:
print(type(train_prompts), type(train_labels))
print(len(train_prompts), len(train_labels))
print(train_labels[0], train_prompts[0][:])

<class 'list'> <class 'list'>
26 26
2 Generate a cover letter responding to the following job application. Be sure to include these skills. 


In [17]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [21]:
train_encodings = tokenizer(train_prompts, truncation=True, padding=True)
val_encodings = tokenizer(val_prompts, truncation=True, padding=True)
# test_encodings = tokenizer(test_prompts, truncation=True, padding=True)

In [22]:
import torch

class PromptsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PromptsDataset(train_encodings, train_labels)
val_dataset = PromptsDataset(val_encodings, val_labels)
# test_dataset = PromptsDataset(test_encodings, test_labels)

In [23]:
train_dataset[5]

{'input_ids': tensor([  101,  4339,  1037,  3104,  3661,  2005,  2033,  1012,  2421,  1996,
          2206,  4813,  1012,  6869,  2000,  1996,  3582,  3105, 14739,  1024,
           102,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0]),
 'labels': tensor(2)}

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [26]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    f"distilledbert-finetuned-prompts",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=5,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    push_to_hub=True,
    # metric_for_best_model="accuracy",
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    # compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

Epoch,Training Loss,Validation Loss
1,No log,1.049725
2,No log,1.049878
3,No log,1.049613
4,1.108900,1.048847
5,1.108900,1.047952


TrainOutput(global_step=15, training_loss=1.1050284067789713, metrics={'train_runtime': 88.8499, 'train_samples_per_second': 1.463, 'train_steps_per_second': 0.169, 'total_flos': 840872506500.0, 'train_loss': 1.1050284067789713, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7034949660301208,
 'eval_runtime': 25.1925,
 'eval_samples_per_second': 1.985,
 'eval_steps_per_second': 0.04,
 'epoch': 3.0}

In [34]:
train_prompts[2]

'Complete this sentence: My technical experience, combined with a growth mindset and passion...'

In [36]:
from transformers import DistilBertTokenizer

# Load the trained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilledbert-finetuned-prompts", num_labels=3)

# Example texts for testing
examples = [
    "Please complete the following sentence:",
    "Please write me a cover letter",
    "Complete this sentence: My technical experience, combined with a growth mindset and passion..."

]

# Tokenize and encode the examples
encoded_inputs = tokenizer(examples, padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    model.eval()
    outputs = model(**encoded_inputs)

# # Get the predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# # Map the predicted labels to their corresponding text
predicted_texts = [NUM_TO_CAT[label] for label in predicted_labels]

# # Print the predicted labels for the examples
for text, label in zip(examples, predicted_texts):
    print(f"Text: {text}\nPredicted Label: {label}\n")


Text: Please complete the following sentence:
Predicted Label: Generate a cover letter with constraints

Text: Please write me a cover letter
Predicted Label: Generate a cover letter with constraints

Text: Complete this sentence: My technical experience, combined with a growth mindset and passion...
Predicted Label: Generate a cover letter with constraints

